到现在为止我们并没有关心过我们的数据来自哪里以及我们所构建的模型是如何部署的，更没有关心过这个模型可能存在很大问题的。许多失败的机器学习模型的部署都可以追溯为这种情况。**但实际上是说我们其实是可以及早发现这些情况从而缓解这些情况的**，发现这些存在潜在的致命问题有时候可能会很简单也可能会很复杂。

## (Covariate Shift)协变量转移

这是一个很容易理解但常常被我们所忽略的问题，当我们辨别猫或是狗的时候，我们的训练集由下面的图片组成：

|![](../img/Chapter2-Supervised-Learning/cat1.jpg)|![](../img/Chapter2-Supervised-Learning/cat2.jpg)|![](../img/Chapter2-Supervised-Learning/dog1.jpg)|![](../img/Chapter2-Supervised-Learning/dog2.jpg)|
|:---------------:|:---------------:|:---------------:|:---------------:|
|cat|cat|dog|dog|

但当我们测试时我们却分类如下的图片：

|![](../img/Chapter2-Supervised-Learning/cat-cartoon1.png)|![](../img/Chapter2-Supervised-Learning/cat-cartoon2.png)|![](../img/Chapter2-Supervised-Learning/dog-cartoon1.png)|![](../img/Chapter2-Supervised-Learning/dog-cartoon2.jpg)|
|:---------------:|:---------------:|:---------------:|:---------------:|
|cat|cat|dog|dog|

很明显我们的预测结果会很差，原因是我们的训练集是由真实图片组成的但是我们的测试集却是一堆卡通图片，单纯地对一个与测试集有很大差异的数据集进行训练，没有任何的改进措施，效果往往很插上，不幸的是，这种陷阱其实很常见。统计学家把这个称为协变量转移，即适合于组成训练数据的分布被转移到了毫不相关的测试集上，从概率的角度来说，关于训练数据的分布$p(x)$改变了或者说转移到了测试数据上但在给定数据下真实标签的分布$p(y|x)$却没有改变。

## Concept Shift(概念转换)

另一个相类似的问题我们成为概念转换，它发生在当我们的对真实标签的说法改变的时候，如果我们建立一个机器翻译系统，那么$p(y|x)$应该有所不同，比如，不同的地区对同一种商品的叫法不尽相同，再比如，NOKIA手机的广告点击率也随地区有所不同，

## Examples

### Medical Diagnostics

Imagine you want to design some algorithm to detect cancer. You get data of healthy and sick people; you train your algorithm; it works fine, giving you high accuracy and you conclude that you’re ready for a successful career in medical diagnostics. Not so fast ...

Many things could go wrong. In particular, the distributions that you work with for training and those in the wild might differ considerably. This happened to an unfortunate startup I had the opportunity to consult for many years ago. They were developing a blood test for a disease that affects mainly older men and they’d managed to obtain a fair amount of blood samples from patients. It is considerably more difficult, though, to obtain blood samples from healthy men (mainly for ethical reasons). To compensate for that, they asked a large number of students on campus to donate blood and they performed their test. Then they asked me whether I could help them build a classifier to detect the disease. I told them that it would be very easy to distinguish between both datasets with probably near perfect accuracy. After all, the test subjects differed in age, hormone level, physical activity, diet, alcohol consumption, and many more factors unrelated to the disease. This was unlikely to be the case with real patients: Their sampling procedure had caused an extreme case of covariate shift that couldn’t be corrected by conventional means. In other words, training and test data were so different that nothing useful could be done and they had wasted significant amounts of money. 

### Self Driving Cars

A company wanted to build a machine learning system for self-driving cars. One of the key components is a roadside detector. Since real annotated data is expensive to get, they had the (smart and questionable) idea to use synthetic data from a game rendering engine as additional training data. This worked really well on 'test data' drawn from the rendering engine. Alas, inside a real car it was a disaster. As it turned out, the roadside had been rendered with a very simplistic texture. More importantly, *all* the roadside had been rendered with the *same* texture and the roadside detector learned about this 'feature' very quickly. 

A similar thing happened to the US Army when they first tried to detect tanks in the forest. They took aerial photographs of the forest without tanks, then drove the tanks into the forest and took another set of pictures. The so-trained classifier worked 'perfectly'. Unfortunately, all it had learned was to distinguish trees with shadows from trees without shadows - the first set of pictures was taken in the early morning, the second one at noon. 

### Nonstationary distributions

A much more subtle situation is where the distribution changes slowly and the model is not updated adequately. Here are a number of typical cases:

* We train a computational advertising model and then fail to update it frequently (e.g. we forget to incorporate that an obscure new device called an iPad was just launched). 
* We build a spam filter. It works well at detecting all spam that we've seen so far. But then the spammers wisen up and craft new messages that look quite unlike anything we've seen before. 
* We build a product recommendation system. It works well for the winter. But then it keeps on recommending Santa hats after Christmas.

### More Anecdotes

* We build a classifier for "Not suitable/safe for work" (NSFW) images. To make our life easy, we scrape a few seedy Subreddits. Unfortunately the accuracy on real life data is lacking (the pictures posted on Reddit are mostly 'remarkable' in some way, e.g. being taken by skilled photographers, whereas most real NSFW images are fairly unremarkable ...). Quite unsurprisingly the accuracy is not very high on real data.
* We build a face detector. It works well on all benchmarks. Unfortunately it fails on test data - the offending examples are close-ups where the face fills the entire image (no such data was in the training set). 
* We build a web search engine for the USA market and want to deploy it in the UK. 

In short, there are many cases where training and test distribution $p(x)$ are different. In some cases, we get lucky and the models work despite the covariate shift. We now discuss principled solution strategies. Warning - this will require some math and statistics.

## Covariate Shift Correction

假设我们想估计一些我们已经标记了的数据$(x_i, y_i)$之间的依赖关系$p(y|x)$，不幸的是，我们的观察值是从分布$q(x)$中产生的而不是从正确的分布$p(x)$中产生的。为了取得进展，我们首先回顾训练过程中究竟发生了什么：我们迭代数据和相关标签$\{(x_1,y_1),...,(x_m,y_m)\}$，然后在每个$mini-batch$后更新我们的权重向量，我们也根据情况对参数添加的$l_2$罚项，即$l_2$正则化，换句话说，我们想要解决以下问题：

$$\mathop{\mathrm{minimize}}_w \frac{1}{m} \sum_{i=1}^m l(x_i, y_i, f(x_i)) + \frac{\lambda}{2} \|w\|_2^2$$

统计学家通常把上式成为**经验平均**，即从分布$p(x)p(y|x)$上采样的数据计算的平均。如果我们的数据是从错误的分布$q$上采样的，我们可以通过下面的等式来进行修正：

$$\mathbf{E}_{x \sim p(x)} [f(x)] = \int f(x) p(x) dx = \int f(x) \frac{p(x)}{q(x)} q(x) dx = \mathbf{E}_{x \sim q(x)} \left[f(x) \frac{p(x)}{q(x)}\right]$$

换句话说，我们需要通过比例系数(加权)为每个样本重新赋予权重，比例系数将从我们修正后的分布$\beta(x) := p(x)/q(x)$中得出，不幸的是，我们并不知道这些比例，因此我需要进行估计，很多方法都可以采用，例如，一些比较花哨的算子理论试图直接用最小范数或最大熵原理来重新校准期望，需要注意的是，使用这些方法我们必须同时知道真实分布$p$以及错误的分布$q$。

在这种情况下，我们可以使用逻辑回归来估计比例系数。我们学习一个分类器来区分从分布$p(x)$和从分布$q(x)$采样的数据。如果不能区分某个数据来自哪个分布，这说明该数据可以来自两者中任意一个分布，这也从另一方面说明，可以被很好区分的数据都应该是over/underweigted的，为了简单起见，假设来自两个分布的实例数目是相等的，即$x_i \sim p(x)$ and $x_i′ \sim q(x)$，我们用$z_i$

## (TODO@ANG)